In [ ]:
from modules.matlab_metrics import *
from modules.image_utils import *
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

tile_path = 'data/toulon-laspezia-tiles-noseaclouds/WV02/train/WV02_La_Spezia_2011_10_28_011650745010_0/pan/00011.tif'
test_img = 'data/PIRM_dataset/Validation/Original/2.png'
#test_img = 'data/test.png'
n=10
mu = 370.93775769104

In [ ]:
img = geotiff_to_ndarray(tile_path)
mu = np.mean(img)
print(img.dtype, np.min(img), np.max(img), np.mean(img))
img_sc = input_scaler(img, radius=1.0, output_dtype='float32', uint_bit_depth=11,
                      mean_correction=True, mean=mu)
print(img_sc.dtype, np.min(img_sc), np.max(img_sc), np.mean(img_sc))
img_unsc = output_scaler(img_sc, radius=1.0, output_dtype='uint16', uint_bit_depth=11,
                         mean_correction=True, mean=mu)
print(img_unsc.dtype, np.min(img_unsc), np.max(img_unsc), np.mean(img_unsc))
plt.imshow(img_sc, cmap='gray')

In [ ]:
imgs = np.stack([img_sc]*n)
imgs.shape
#imgs = imgs.astype('uint8')
#imgs.shape
#imgs.dtype


In [ ]:
matlab_engine = MatLabEngine('modules/matlab', ma=True, niqe=True, 
                             input_range=(-1,1), output_dtype='uint16', output_bit_depth=11,
                             scale_mean=mu, stretch=False, shave_width=4)

In [ ]:
tic = time.perf_counter()
niqes = matlab_engine.matlab_niqe_metric(imgs)
toc = time.perf_counter()
print('niqe', niqes)
print(toc-tic, 's')
print((toc-tic)/n, 's per img')

In [ ]:
tic = time.perf_counter()
ma = matlab_engine.matlab_ma_metric(imgs)
toc = time.perf_counter()
print('ma', ma)
print(toc-tic, 's')
print((toc-tic)/n, 's per img')

In [ ]:
a = tf.constant(0)
type(a)
a=2
tf.is_tensor(a)